In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/upcoming-event.csv')
len(df)

4256

In [6]:
num_upcoming_fights = len(df_upcoming)

In [7]:
df = df_upcoming.append(df)
len(df)

4269

In [8]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [10]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [11]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

13
13
13
4256
4256
4256


In [12]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,label
0,Dustin Poirier,Dan Hooker,-225,180,44.444444,180.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Red,0.0
1,Mike Perry,Mickey Gall,-315,235,31.746032,235.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0.0
2,Brendan Allen,Kyle Daukaus,-305,235,32.786885,235.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,0.0
3,Gian Villante,Maurice Greene,180,-230,180.000000,43.478261,2020-06-27,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1.0
4,Luis Pena,Khama Worthy,-250,195,40.000000,195.000000,2020-06-27,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4251,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,1.0
4252,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0.0
4253,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0.0
4254,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,label
0,Kamaru Usman,Jorge Masvidal,-250,200,40.000000,200.000000,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,Red,NaN
1,Alexander Volkanovski,Max Holloway,-230,185,43.478261,185.000000,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,9.0,Red,NaN
2,Petr Yan,Jose Aldo,-250,200,40.000000,200.000000,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,13.0,6.0,NaN,NaN,Red,NaN
3,Jessica Andrade,Rose Namajunas,180,-225,180.000000,44.444444,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,Red,NaN
4,Amanda Ribas,Paige VanZant,-835,525,11.976048,525.000000,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN
5,Volkan Oezdemir,Jiri Prochazka,-177,140,56.497175,140.000000,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN
6,Elizeu Zaleski dos Santos,Muslim Salikhov,108,-134,108.000000,74.626866,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN
7,Makwan Amirkhani,Danny Henry,-215,170,46.511628,170.000000,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN
8,Leonardo Santos,Roman Bogatov,-190,150,52.631579,150.000000,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN
9,Marcin Tybura,Maxim Grishin,-112,-112,89.285714,89.285714,2020-07-11,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN


In [13]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
        
        print()
        
    return(0)
        

In [14]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]

    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    
    return(0)

In [15]:
with open('../automated_model_creation/models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice:
model_choice = 7

print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

[['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_06_13_20', 'model_06_20_20', 'model_6_27_20'], ["LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n                   intercept_scaling=1, l1_ratio=None, max_iter=50000,\n                   multi_class='auto', n_jobs=None, penalty='l2',\n                   random_state=75, solver='lbfgs', tol=5.993308915320019,\n                   verbose=0, warm_start=False)", "DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=264,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')", "RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,\n                       criterion='ent

In [16]:

get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Kamaru Usman vs. Jorge Masvidal
Kamaru Usman has an EV of 35.6 on odds of -250.  They have 96.85 % chance of winning.

Alexander Volkanovski vs. Max Holloway
Alexander Volkanovski has an EV of 30.91 on odds of -230.  They have 91.24 % chance of winning.

Petr Yan vs. Jose Aldo
Petr Yan has an EV of 34.82 on odds of -250.  They have 96.3 % chance of winning.

Jessica Andrade vs. Rose Namajunas
Jessica Andrade has an EV of 30.45 on odds of 180.  They have 46.59 % chance of winning.

Amanda Ribas vs. Paige VanZant
Amanda Ribas has an EV of 10.32 on odds of -835.  They have 98.52 % chance of winning.

Volkan Oezdemir vs. Jiri Prochazka
Jiri Prochazka has an EV of 27.64 on odds of 140.  They have 53.18 % chance of winning.

Elizeu Zaleski dos Santos vs. Muslim Salikhov
Elizeu Zaleski dos Santos has an EV of 12.94 on odds of 108.  They have 54.3 % chance of winning.

Makwan Amirkhani vs. Danny Henry
Makwan Amirkhani has an EV of 24.02 on odds of -215.  They have 84.65 % chance of winning.

L

0